In [1]:
from os.path import join
import numpy as np
import pandas as pd
import torch
import torch.multiprocessing as mp
from astropy.io import fits

from networks import ForkCNN
from train import train_nn, prepare_dataloader, ddp_setup
from dataset import FiberDataset
import config

fits_dir = '/xdisk/timeifler/wxs0703/kl_nn/fits/'
samp_dir = '/xdisk/timeifler/wxs0703/kl_nn/samples/samples.csv'
fig_dir = '/xdisk/timeifler/wxs0703/kl_nn/figures/'

In [2]:
world_size = torch.cuda.device_count()
save_every = 1
nepochs = config.train['epoch_number']
batch_size = config.train['batch_size']
nfeatures = config.train['feature_number']
f_valid = config.train['validation_split']

In [3]:
mp.spawn(train_nn, args=(world_size, save_every, nepochs, batch_size, nfeatures, f_valid), nprocs=world_size)

/home/u2/wxs0703/.local/lib/python3.9/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/u2/wxs0703/.local/lib/python3.9/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
INFO:Trainer:[TRAIN] Epoch: 1 Loss: 0.5708736422848625 Time: 1:6
INFO:Trainer:[VALID] Epoch: 1 Loss: 0.5310364911243866 Time: 0:7
INFO:Trainer:[TRAIN] Epoch: 2 

KeyboardInterrupt: 

In [3]:
samples = pd.read_csv(samp_dir)
ranges = config.par_ranges
print(ranges)

{'g1': [-1, 1], 'g2': [-1, 1], 'theta_int': [-3.141592653589793, 3.141592653589793], 'sini': [0, 1], 'v0': [-30, 30], 'vcirc': [60, 540], 'rscale': [0.1, 10], 'hlr': [0.1, 5]}


In [4]:
samples

,ID,g1,g2,theta_int,sini,v0,vcirc,rscale,hlr
0,0,-0.067114,0.109753,-1.315385,0.28175,-21.627,482.040,1.550845,4.050135
1,1,0.920613,-0.272803,1.735730,0.28755,-1.203,175.848,9.423325,4.075125
2,2,-0.065653,-0.429348,-1.163332,0.42925,17.643,429.576,2.263645,0.675015
3,3,-0.393528,-0.552889,2.840314,0.28665,-6.669,227.688,2.423035,4.975745
4,4,0.923271,-0.042668,-1.792279,0.48335,-6.495,175.704,6.936445,3.292595
...,...,...,...,...,...,...,...,...,...
9995,9995,-0.289780,-0.840344,-0.854199,0.16495,23.817,457.800,6.810715,4.996325
9996,9996,0.358207,0.179549,-3.129340,0.21495,-14.667,437.688,5.492035,1.521735
9997,9997,-0.072135,0.449051,2.426252,0.76325,-1.377,455.400,9.077815,2.435095
9998,9998,0.421660,-0.251699,0.771889,0.74685,21.183,189.192,2.064655,2.212635


In [5]:
for par, values in samples.items():
    if par != 'ID':
        low, high = ranges[par]
        values -= low
        values /= high-low

In [6]:
np.array(samples.iloc[0])[1:]

array([0.46644282, 0.55487637, 0.29065   , 0.28175   , 0.13955   ,
       0.87925   , 0.14655   , 0.80615   ])